In [1]:
import anndata as ad
import pandas as pd
import pickle as pkl
import random

In [2]:
OBS_FILE = "C:\\Users\\curea\\Documents\\bioFM for drug discovery\\dege-fm\\raw-data\\sciplex_obs_annot_with_smiles_manual_plus_embeddings.tsv"
EMB_FILE = "C:\\Users\\curea\\Documents\\bioFM for drug discovery\\dege-fm\\raw-data\\uce_embeddings.pkl"

adata_obs = pd.read_csv(OBS_FILE, sep="\t")

#clean redundant columns
cols_to_remove = ["Unnamed: 0.2", "Unnamed: 0.1", "Unnamed: 0"]
adata_obs = adata_obs.drop(columns=cols_to_remove)

with open(EMB_FILE, 'rb') as f:
    adata_X = pkl.load(f)

#create adata object for ease of use
adata = ad.AnnData(X=adata_X, obs=adata_obs)

# remove cells with a mixture of compounds treatement

adata = adata[adata.obs['sm_embedding'] != "MIXTURE_OF_COMPOUNDS"]


C:\Users\curea\anaconda3\envs\stable_diffusion\Lib\site-packages\anndata\_core\aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [3]:
adata.obs['product_name'].unique()

array(['Enzastaurin (LY317615)', 'Raltitrexed', 'Vehicle',
       'Lenalidomide (CC-5013)', 'MLN8054', 'Sodium Phenylbutyrate',
       'Celecoxib', 'Linifanib (ABT-869)', 'Thalidomide', 'SNS-314',
       '(+)-JQ1', 'S3I-201', 'AZD1480', 'PF-573228', 'CEP-33779',
       'Resminostat', 'Tie2 kinase inhibitor', 'Nintedanib (BIBF 1120)',
       'SRT3025 HCl', 'Avagacestat (BMS-708163)',
       'Quisinostat (JNJ-26481585) 2HCl', 'MC1568',
       'Temsirolimus (CCI-779, NSC 683864)', 'IOX2',
       'Bosutinib (SKI-606)', 'Regorafenib (BAY 73-4506)',
       'Aurora A Inhibitor I', 'ENMD-2076', 'Curcumin', 'NVP-BSK805 2HCl',
       'Tozasertib (VX-680, MK-0457)', 'Veliparib (ABT-888)',
       'Crizotinib (PF-02341066)', 'Panobinostat (LBH589)',
       'Alvespimycin (17-DMAG) HCl', 'Sorafenib Tosylate', 'Maraviroc',
       'Baricitinib (LY3009104, INCB028050)', 'Mesna ',
       'Fedratinib (SAR302503, TG101348)', 'Trametinib (GSK1120212)',
       'Roscovitine (Seliciclib,CYC202)', 'S-Ruxolitini

In [4]:
def split_compounds_train_test(compound_list, control_name="Vehicle", split_ratio=0.7):

    if control_name not in compound_list:
        raise ValueError("The control values are not in the list.")

    compound_list = [elem for elem in compound_list if elem != control_name]
    random.shuffle(compound_list)

    total_size = len(compound_list) + 1
    split_size_train = int(split_ratio * total_size)

    # Split the list into 30% and 70%
    split_train = compound_list[:split_size_train ]
    split_test = compound_list[split_size_train :]

    # Add vehicle in both lists
    split_train.append(control_name)
    split_test.append(control_name)

    return split_train, split_test

In [5]:
compound_list = list(adata.obs['product_name'].unique())
cp_train, cp_test = split_compounds_train_test(compound_list, control_name="Vehicle", split_ratio=0.7)

adata_train = adata[adata.obs['product_name'].isin(cp_train)]
adata_test = adata[adata.obs['product_name'].isin(cp_test)]

In [6]:
adata_train

View of AnnData object with n_obs × n_vars = 408498 × 1280
    obs: 'index', 'cell_type', 'dose', 'dose_character', 'dose_pattern', 'g1s_score', 'g2m_score', 'pathway', 'pathway_level_1', 'pathway_level_2', 'product_dose', 'product_name', 'proliferation_index', 'replicate', 'size_factor', 'target', 'vehicle', 'label', 'pubchem_name', 'pubchem_ID', 'smiles', 'sm_embedding'

In [7]:
adata_test

View of AnnData object with n_obs × n_vars = 176412 × 1280
    obs: 'index', 'cell_type', 'dose', 'dose_character', 'dose_pattern', 'g1s_score', 'g2m_score', 'pathway', 'pathway_level_1', 'pathway_level_2', 'product_dose', 'product_name', 'proliferation_index', 'replicate', 'size_factor', 'target', 'vehicle', 'label', 'pubchem_name', 'pubchem_ID', 'smiles', 'sm_embedding'

In [8]:
adata_train.write_h5ad("C:\\Users\\curea\\Documents\\bioFM for drug discovery\\dege-fm\\data\\adata_train_v1.h5ad")

adata_test.write_h5ad("C:\\Users\\curea\\Documents\\bioFM for drug discovery\\dege-fm\\data\\adata_test_v1.h5ad")

C:\Users\curea\anaconda3\envs\stable_diffusion\Lib\site-packages\anndata\_core\anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
C:\Users\curea\anaconda3\envs\stable_diffusion\Lib\site-packages\anndata\_core\anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
C:\Users\curea\anaconda3\envs\stable_diffusion\Lib\site-packages\anndata\_core\anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
C:\Users\curea\anaconda3\envs\stable_diffusion\Lib\site-packages\anndata\_core\anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
C:\Users\curea\anaconda3\envs\stable_diffusion\Lib\site-packages\anndata\_core\anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.ob